In [89]:
# Ignore the warnings
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
train = pd.read_csv("../asset/train.csv")
test = pd.read_csv("../asset/test.csv")
weather = pd.read_csv("../asset/weather.csv")
key = pd.read_csv("../asset/key.csv")
submission_example = pd.read_csv("../asset/sampleSubmission.csv")

# Success - Display the first record
print("Train data : ", train.shape)
print("Test  data : ", test.shape)
print("weather  data : ", weather.shape)
print("key  data : ", key.shape)
print("submission  data : ", submission_example.shape)

coulmns_should_be_integer_type_of_data = ["tmax", "tmin", "tavg", "dewpoint", "wetbulb", "heat", "cool"]
coulmns_should_be_float_type_of_data = ["snowfall", "preciptotal", "stnpressure", "sealevel", "depart",\
                                        "resultspeed", "resultdir", "avgspeed"]

import pickle
import awesome_functions as cf

slack_url = pickle.load(open("send_url.pickle", "rb"))
slack_url

# 원본을 유지하기 위해서 카피
df_train = train.copy()
df_weather = weather.copy()
df_key = key.copy()
df_train_with_weather = pd.read_csv("df_train_with_station_nbr_201806202007.csv")

Train data :  (4617600, 4)
Test  data :  (526917, 3)
weather  data :  (20517, 20)
key  data :  (45, 2)
submission  data :  (526917, 2)


In [93]:
import datetime

In [157]:
def addDays(date):
    temp = date.split("-")
    year, month, date = int(temp[0]), int(temp[1]), int(temp[2])
    return datetime.datetime(year, month, date).weekday() + 1
df_train_with_weather["day"] = df_train_with_weather["date"].apply(addDays)

In [160]:
def addDays(date):
    temp = date.split("-")
    year, month, date = int(temp[0]), int(temp[1]), int(temp[2])
    return datetime.datetime(year, month, date).weekday()

def changeToStr(day):
    return str(day)

In [161]:
df_train_with_weather["isWeekend"] = df_train_with_weather["day"].apply(addIsWeekend)

In [162]:
def changeToStr(day):
    return str(day)

In [163]:
df_train_with_weather["day"] = df_train_with_weather["day"].apply(changeToStr)

In [164]:
df = df_train_with_weather.copy()

In [165]:
df.tail()

,date,store_nbr,item_nbr,units,station_nbr,day,isWeekend
4617595,2014-10-31,45,107,0,16,5,0
4617596,2014-10-31,45,108,0,16,5,0
4617597,2014-10-31,45,109,0,16,5,0
4617598,2014-10-31,45,110,0,16,5,0
4617599,2014-10-31,45,111,0,16,5,0


In [5]:
def getStoreList(df, item_nbr, msg=True):
    df_ = df.copy()
    df_ = df_[df_["item_nbr"] == item_nbr]
    df_ = df_[df_["units"] != 0]
    if msg:
        print("팔린 개수가 0이상인 row 개수 : " + str(len(df_)))
        print("총 팔린 개수 : " + str(df_["units"].sum()))
        print(str(item_nbr) + "번 아이템이 팔린 스토어 개수 : " + str(len(list(df_["store_nbr"].unique()))))
        print(str(item_nbr) + "번 아이템이 팔린 스토어 넘버 리스트 : " + str(list(df_["store_nbr"].unique())))
#     len(df_)
#     str(df_["units"].sum())
    return list(df_["store_nbr"].unique())

In [83]:
def report_item_sales(df):
    item_nbr_li = df["item_nbr"].unique()
    for item_nbr in item_nbr_li:
        getStoreList(df, item_nbr)

In [108]:
def getDf(df, store_nbr_li, item_nbr):
    li = []
    for store in store_nbr_li:
        li.append((store, item_nbr))
    mask_train = [(sno_ino in li) for sno_ino in zip(df['store_nbr'], df['item_nbr'])]
    df_ = df[mask_train].copy()
    return df_

In [166]:
df_ = getDf(df, range(1, 46), 45)

In [167]:
df_.reset_index(drop=True, inplace=True)

In [168]:
df_.tail()

,date,store_nbr,item_nbr,units,station_nbr,day,isWeekend
15116,2014-10-31,34,45,45,17,5,0
15117,2014-10-31,37,45,20,13,5,0
15118,2014-10-31,38,45,63,14,5,0
15119,2014-10-31,40,45,44,13,5,0
15120,2014-10-31,42,45,35,14,5,0


In [169]:
df_merged_weather = df_.merge(df_weather, on=["date", "station_nbr"], how="inner")

In [170]:
cf.remove_columns(df_merged_weather, ["depart", "snowfall", "sunrise", "sunset"])

제거한 컬럼명 : ['depart', 'snowfall', 'sunrise', 'sunset']


,date,store_nbr,item_nbr,units,station_nbr,day,isWeekend,tmax,tmin,tavg,...,wetbulb,heat,cool,codesum,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
15116,2014-10-31,25,45,24,13,5,0,58,35,47,...,37,18,0,,0.00,24.36,30.18,6.1,17,7.2
15117,2014-10-31,32,45,18,13,5,0,58,35,47,...,37,18,0,,0.00,24.36,30.18,6.1,17,7.2
15118,2014-10-31,37,45,20,13,5,0,58,35,47,...,37,18,0,,0.00,24.36,30.18,6.1,17,7.2
15119,2014-10-31,40,45,44,13,5,0,58,35,47,...,37,18,0,,0.00,24.36,30.18,6.1,17,7.2
15120,2014-10-31,29,45,26,3,5,0,55,33,44,...,41,21,0,,0.00,29.65,30.36,11.1,35,11.5


In [171]:
cf.isThereNoneData(df_merged_weather)

M Data over 60% : []
T Data over 60% : []
B Data over 60% : []


,Column,Row Count,Missing Data,M Data %,Trace Data,T Data %,Bar Data,B Data %
0,date,15121,0,0.00,0,0.00,0,0.0
1,store_nbr,15121,0,0.00,0,0.00,0,0.0
2,item_nbr,15121,0,0.00,0,0.00,0,0.0
3,units,15121,0,0.00,0,0.00,0,0.0
4,station_nbr,15121,0,0.00,0,0.00,0,0.0
5,day,15121,0,0.00,0,0.00,0,0.0
6,isWeekend,15121,0,0.00,0,0.00,0,0.0
7,tmax,15121,84,0.56,0,0.00,0,0.0
8,tmin,15121,82,0.54,0,0.00,0,0.0
9,tavg,15121,185,1.22,0,0.00,0,0.0


In [178]:
cf.remove_columns(df_merged_weather, ["tmax", "tavg"])

제거한 컬럼명 : ['tmax', 'tavg']


,units,day,isWeekend,tmin,dewpoint,wetbulb,heat,cool,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
15116,24,5,0,35,28,37,18,0,0.00,24.36,30.18,6.1,17,7.2
15117,18,5,0,35,28,37,18,0,0.00,24.36,30.18,6.1,17,7.2
15118,20,5,0,35,28,37,18,0,0.00,24.36,30.18,6.1,17,7.2
15119,44,5,0,35,28,37,18,0,0.00,24.36,30.18,6.1,17,7.2
15120,26,5,0,33,32,41,21,0,0.00,29.65,30.36,11.1,35,11.5


In [179]:
df_merged_weather_copy = df_merged_weather.copy()

In [180]:
df_merged_weather_copy = cf.remove_m_row(df_merged_weather_copy)

원본 데이터프레임 length : 15121
제거하고 싶은 컬럼 : ['units', 'day', 'isWeekend', 'tmin', 'dewpoint', 'wetbulb', 'heat', 'cool', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir', 'avgspeed']
제거한 int 컬럼 : ['tmin', 'dewpoint', 'wetbulb', 'heat', 'cool']
제거한 float 컬럼 : ['preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir', 'avgspeed']
제거한 후 데이터프레임 length : 14793


In [181]:
df_merged_weather_copy = df_merged_weather_copy.drop("isWeekend", axis = 1)

In [151]:
def getFormula(df, categorical_columns, target_column):
    """
        df : 회귀분석할 df
        categorical_columns : 카테고리 컬럼명 리스트
        target_column : target column명 문자열
    """
    columns = df.columns
    formula = target_column + " ~ "
    for i, column in enumerate(columns):
        if column != target_column:
            if i != 0:
                formula += " + "
            if column in categorical_columns:
                formula += "C(" + column + ")"
            else:
                formula += str(column)
    return formula

In [186]:
models = sm.OLS.from_formula(getFormula(df_merged_weather_copy, ["day"], "units") + " + 0", data=df_merged_weather_copy)

In [187]:
result = models.fit()

In [127]:
len(dfX_)

14793

In [128]:
len(dfY)

14793

In [129]:
dfY.sum()

984679

In [190]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  units   R-squared:                       0.132
Model:                            OLS   Adj. R-squared:                  0.131
Method:                 Least Squares   F-statistic:                     132.6
Date:                Sun, 01 Jul 2018   Prob (F-statistic):               0.00
Time:                        19:25:15   Log-Likelihood:                -69207.
No. Observations:               14793   AIC:                         1.384e+05
Df Residuals:                   14775   BIC:                         1.386e+05
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
C(day)[1]     331.6872     43.216      7.675      0.000     246.978     416.397
C(day)[2]     324.5843     43.214      7.511      0.000     239.880     409.289
C(day)[3]     322.9924     43.213      7.474      0.000     238.290     407.694
C(day)[4]     319.6299     43.221      7.395      0.000     234.912     404.348
C(day)[5]     329.4058     43.218      7.622      0.000     244.694     414.118
C(day)[6]     341.1139     43.207      7.895      0.000     256.422     425.805
C(day)[7]     349.0466     43.201      8.080      0.000     264.367     433.726
tmin            0.1352      0.068      1.981      0.048       0.001       0.269
dewpoint       -0.0562      0.083     -0.680      0.496      -0.218       0.106
wetbulb        -0.2353      0.184     -1.278      0.201      -0.596       0.126
heat            0.1008      0.111      0.908      0.364      -0.117       0.318
cool            0.4039      0.107      3.782      0.000       0.195       0.613
preciptotal     0.2256      1.179      0.191      0.848      -2.085       2.537
stnpressure     0.6203      0.128      4.856      0.000       0.370       0.871
sealevel       -9.2049      1.431     -6.434      0.000     -12.009      -6.400
resultspeed     0.0356      0.116      0.306      0.760      -0.192       0.263
resultdir       0.0031      0.024      0.132      0.895      -0.043       0.049
avgspeed       -0.1531      0.127     -1.204      0.229      -0.402       0.096
==============================================================================
Omnibus:                     2740.341   Durbin-Watson:                   1.244
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8342.612
Skew:                           0.961   Prob(JB):                         0.00
Kurtosis:                       6.138   Cond. No.                     5.06e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.06e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""